Load the data from the csv file into a Pandas dataframe  
Delete any NULL addresses  
Add a new column called street which will hold the street names only (without the house number)

In [15]:
import pandas
import re
from numpy import NaN

data = pandas.read_csv( 'premises-list-as-at-8-february-2021.csv', encoding= 'unicode_escape', usecols=['Address'] )
data.dropna( inplace=True)
data['Street'] = None
data

,Address,Street
0,1A Barker Ave,None
1,12 Burkinshaw St,None
2,36-38 River St,None
3,Beach St,None
4,26 Humphries Rd,None
...,...,...
18742,UNIT 4 6 LEIGHTON PL,None
18743,13A LEWIS ST,None
18744,271 Powder Works Road,None
18745,160 PRINCES HWY,None


Create a road types lookup dictionary to us to make sure all road types are cleaned up with a standard name  
(road_types.lookup was created from the clean_roads Jupyter notebook)

In [16]:
road_type_lookup = {}

with open( 'road_types.lookup', 'r') as lookup:
    for line in lookup:
        lookup, abbreviation = line.strip().split(',')
        road_type_lookup[lookup] = abbreviation

road_type_lookup



{'ACCS': 'ACCESS',
 'ARCADE': 'ARC',
 'AV': 'AVE',
 'AVENUE': 'AVE',
 'AVENUES': 'AVE',
 'AVES': 'AVE',
 'BLVDE': 'BLVD',
 'BLVE': 'BLVD',
 'BOULEVARD': 'BLVD',
 'BOULEVARDE': 'BLVD',
 'BVD': 'BLVD',
 'BVDE': 'BLVD',
 'CHASE': 'CH',
 'CIRCUIT': 'CCT',
 'CLOSE': 'CL',
 'COURT': 'CT',
 'COURTS': 'CT',
 'COVE': 'CV',
 'CRES': 'CR',
 'CRESCENT': 'CR',
 'CRESENT': 'CR',
 'CROSSING': 'CROSS',
 'CRT': 'CT',
 'DRIBE': 'DR',
 'DRIVE': 'DR',
 'DRIVES': 'DR',
 'DRIVEWAY': 'DR',
 'DRS': 'DR',
 'ESPLANADE': 'ESP',
 'GARDEN': 'GDN',
 'GARDENS': 'GDN',
 'GDNS': 'GDN',
 'GLADE': 'GLDE',
 'GRANGE': 'GRA',
 'GRAOUND': 'GROUND',
 'GROUNDS': 'GROUND',
 'GROVE': 'GR',
 'HEIGHTS': 'HTS',
 'HIGHWAY': 'HWY',
 'HIGHWAY)': 'HWY',
 'HIGHYWAY': 'HWY',
 'HWAY': 'HWY',
 'HWYS': 'HWY',
 'HY': 'HWY',
 'ISLAND': 'ISL',
 'LANE': 'LN',
 'NTH': 'NORTH',
 'OVAL)': 'OVAL',
 'PARADE': 'PDE',
 'PARKWAY': 'PWY',
 'PATHWAY': 'PWAY',
 'PLACE': 'PL',
 'PLAZA': 'PLZA',
 'POINT': 'PT',
 'PROMENADE': 'PROM',
 'QUAY': 'QYS',
 'RD.':

Function to extract the street name from an address
Addressess can take many forms, for example

 * 1 Smith St
 * 1A Smith St
 * Smith St
 * UNIT 1 Smith St

 It also does a lookup to the road type to make sure the road type is using a standard name and cleans up the data

 The logic goes through the address one word at a time in reverse order until it finds 

 * A word containing a number
 * A single letter word (eg A Smith St)
 * A UNIT or Lot name

 Once it finds that position it will add all the remaining words to a string and return it

In [17]:
def get_street_name( address ):
    parts = address.split(' ')

    #  Loop each part and ignore it if its a number or contains a number
    #  or is only one character or is called UNIT or Lot
    #  Once we find one thats not we just get all the rest
    start_index = -1
    num_parts = len( parts )
    times = 0

    while times < num_parts:
        part = parts[start_index].upper()

        #  If first time then check if the road type needs to be cleaned
        #  using the road_type_lookup
        if times == 0:
            if part in road_type_lookup:
                part = road_type_lookup[part]
            parts[start_index] = part

   
        if part.isnumeric() or bool(re.search(r'\d', part)) \
           or len(part ) == 1 or part.upper() == 'UNIT' \
           or part.upper() == 'LOT':
            break
            
        start_index = start_index - 1
        times +=1 

    return ' '.join( parts[start_index+1:] )

Process all the addresses and create the street name using the get_street_name function above  
Add the street name to the 'Street' column

In [18]:
for ind in data.index:
    address = data['Address'][ind]
    
    if not address is NaN:
        street = get_street_name( address )
        data['Street'][ind] = street

data

,Address,Street
0,1A Barker Ave,Barker AVE
1,12 Burkinshaw St,Burkinshaw ST
2,36-38 River St,River ST
3,Beach St,Beach ST
4,26 Humphries Rd,Humphries RD
...,...,...
18742,UNIT 4 6 LEIGHTON PL,LEIGHTON PL
18743,13A LEWIS ST,LEWIS ST
18744,271 Powder Works Road,Powder Works RD
18745,160 PRINCES HWY,PRINCES HWY


Query for all streets that have a length of more than 10
and count how many there are in each group

In [19]:
result=data.query('Street.str.len() > 10')['Street'].value_counts(sort=True, ascending=False)
result

Pacific HWY                                           216
Princes HWY                                           164
Pittwater RD                                           83
Military RD                                            79
Victoria ST                                            72
                                                     ... 
homebush recreation reserve ivanhoe RD                  1
Belrose Community Centre (also Forest Lions CLUBRO      1
Falconer RD                                             1
Alamein Avenue NORTH                                    1
old gosford RD                                          1
Name: Street, Length: 4077, dtype: int64

Get the street with the most number of counts

In [20]:
print(f'The most common street name that has more than 10 letters is {result.nlargest(1).index[0]}')

The most common street name that has more than 10 letters is Pacific HWY
